* Generating Tables for Spec Sources in the Southern Hemisphere after querying the chisqs of the rerun posteriors (Tables 1 and 2)

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import joblib
import h5py
from astropy.coordinates import SkyCoord, match_coordinates_sky
import astropy.units as u

from astropy.io import fits
from functools import reduce
from astropy.table import Table

sys.path.append('../../methods_code_Nresol/')
sys.path.append('../../utils/')
import astro_cuts
import importlib
import spec_utils

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [3]:
fname = 'lsd_scripts/spectrain_south_r50.fits'
df_allspec = spec_utils.convert_to_dataframe_specmatched(fname)
df_goodspec = spec_utils.return_goodspec(df_allspec)

/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/spectra_matched/spec_utils.py:41: RuntimeWarning: divide by zero encountered in true_divide
  df['reduced_chisq'] = df['chisq'].to_numpy() * n_passbands / (n_passbands - 4)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/spectra_matched/spec_utils.py:45: RuntimeWarning: divide by zero encountered in power
  sdss_flux_sig = np.power(np.array(dat['sdss_dr14_starsweep.psfflux_ivar']), -0.5)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/spectra_matched/spec_utils.py:47: RuntimeWarning: divide by zero encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log10(np.array(dat['sdss_dr14_starsweep.psfflux'])[:, ib]), 0.0, np.inf)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/spectra_matched/spec_utils.py:47: RuntimeWarning: invalid value encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log10(np.array(dat['sdss_dr14_starsweep.psfflux'])[:, ib]),

Number of objects before / after 182367 65377
c1 = 0.875, c2 = 0.995, c3 = 0.454
Fraction left = 0.358


In [4]:
len(df_goodspec)

65377

In [5]:
#get tiledict
import healpy as hp
pix256n = hp.ang2pix(256, df_goodspec['l'].to_numpy(), df_goodspec['b'].to_numpy(), lonlat=True, nest=True)
pix256n = np.unique(pix256n)
inputdir = '/n/holylfs05/LABS/finkbeiner_lab/Lab/nmudur/bayestar_edr3/input_fullsky/south/postprocessed/'
dirfiles = np.array(os.listdir(inputdir))
dirfiles = dirfiles[np.array([dirn.endswith('.h5') for dirn in dirfiles])]

In [6]:
dictmapper = spec_utils.get_pix_to_filemapper(inputdir, dirfiles, pix256n)
file2keys = spec_utils.invert_dict(dictmapper)
df_goodspec['pix256n'] = hp.ang2pix(256, df_goodspec['l'].to_numpy(), df_goodspec['b'].to_numpy(), lonlat=True, nest=True)

<ipython-input-6-c6888067c92d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_goodspec['pix256n'] = hp.ang2pix(256, df_goodspec['l'].to_numpy(), df_goodspec['b'].to_numpy(), lonlat=True, nest=True)


In [7]:
df_goodspec['pix256n']

1         623214
2         623215
3         623215
5         623215
6         623215
           ...  
182359    624705
182360    624705
182361    624704
182364    624704
182366    624704
Name: pix256n, Length: 65377, dtype: int64

In [8]:
chisqrerun = np.ones(len(df_goodspec))*np.nan

In [9]:
'NumFiles', len(list(file2keys.keys()))

('NumFiles', 188)

In [10]:
for key in list(file2keys.keys()):
    #key = 'north.00475.h5'
    #opening only relevant files
    print(key)
    inputf = h5py.File(inputdir+key, 'r')
    allcatsrcs = []
    for pixel in file2keys[key]:#for each specmatched pixel
        dat = inputf[f'photometry/pixel 256-{pixel}'] #allcatdata
        allcatsrcs.append(np.vstack([dat['gaia.source_id'], dat['l'], dat['b'], dat['chisq']]).T)
    tab = np.vstack(allcatsrcs)
    specmask = df_goodspec['pix256n'].isin(file2keys[key]).to_numpy() 
    #all specmatsrcs in the specmatched pixels in that file
    specsrcs = df_goodspec.iloc[specmask]
    speclb = SkyCoord(l=specsrcs['l'].to_numpy()*u.deg, b= specsrcs['b'].to_numpy()*u.deg, frame='galactic')
    catlb = SkyCoord(l=tab[:, 1]*u.deg, b= tab[:, 2]*u.deg, frame='galactic')
    ispecincat, d2d, d3d = match_coordinates_sky(speclb, catlb)
    print(np.equal(tab[ispecincat, 0], specsrcs['gaia_edr3.source_id'].to_numpy()).sum()/len(ispecincat))
    chisqrerun[specmask] = tab[ispecincat, -1]
    inputf.close()

south.00599.h5
1.0
south.00709.h5
1.0
south.00658.h5
1.0
south.00539.h5
1.0
south.00577.h5
1.0
south.00605.h5
1.0
south.00588.h5
1.0
south.00675.h5
1.0
south.00067.h5
1.0
south.00058.h5
1.0
south.00102.h5
1.0
south.00861.h5
0.9991503823279524
south.00553.h5
0.9987492182614134
south.00669.h5
1.0
south.00578.h5
1.0
south.00038.h5
1.0
south.00592.h5
1.0
south.00014.h5
1.0
south.01066.h5
1.0
south.00636.h5
1.0
south.00830.h5
1.0
south.00095.h5
0.9966555183946488
south.00670.h5
1.0
south.00557.h5
1.0
south.00862.h5
1.0
south.00667.h5
1.0
south.00559.h5
0.9974489795918368
south.00547.h5
1.0
south.00056.h5
0.9975062344139651
south.00575.h5
1.0
south.00050.h5
1.0
south.00665.h5
1.0
south.00068.h5
1.0
south.00079.h5
1.0
south.00075.h5
1.0
south.00053.h5
1.0
south.00558.h5
0.9989847715736041
south.00013.h5
1.0
south.00556.h5
1.0
south.00076.h5
1.0
south.00054.h5
0.9974358974358974
south.00099.h5
1.0
south.00070.h5
1.0
south.00663.h5
0.9946380697050938
south.00674.h5
1.0
south.00664.h5
1.0
south.

In [11]:
df_goodspec['chisq_rerun'] = chisqrerun
df_goodspec.filter(items=['chisq', 'chisq_rerun'])

<ipython-input-11-9f784ebec496>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_goodspec['chisq_rerun'] = chisqrerun


,chisq,chisq_rerun
1,0.205420,0.205420
2,3.973713,3.973506
3,0.094811,0.059074
5,0.441718,0.442534
6,0.190907,0.191089
...,...,...
182359,0.250157,0.229207
182360,0.106057,0.105947
182361,0.357303,0.355262
182364,0.073998,0.052579


In [12]:
df_goodspec.to_pickle('specmatched_southrerun.pkl')

### Reset with new chisq

In [13]:
df_goodspec['chisq'] = df_goodspec['chisq_rerun']

<ipython-input-13-13dcd7c4c576>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_goodspec['chisq'] = df_goodspec['chisq_rerun']


In [14]:
spec_utils.print_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'])

Number of Stars = 55474, Fraction(%) = 84.8525, Ratio(obj:star)*100 = 100
Number of Galaxies = 87, Fraction(%) = 0.1331, Ratio(obj:star)*100 = 0.1568
Number of QSOs = 9816, Fraction(%) = 15.0145, Ratio(obj:star)*100 = 17.6948


In [3]:
nbtest_dir = '../../notebooks_test/'

def combine_cuts(df_input, cuts_list):
    cutfuncs = [getattr(astro_cuts, ctup[0]) for ctup in cuts_list]
    masklist = []
    for c, ctup in enumerate(cuts_list):
        if len(ctup)==1:
            masklist.append(cutfuncs[c](df_input))
        else:
            masklist.append(cutfuncs[c](df_input, **ctup[1]))
        print('Cut{} {} {}'.format(c, np.sum(masklist[c]), np.sum(masklist[c])/len(masklist[c])))
    final_cut = reduce(np.logical_and, masklist) #intersection of all cuts
    print('Effective Cut = {:.3f}'.format(np.sum(final_cut)/len(final_cut)))
    return final_cut

In [16]:
wisemodel = joblib.load('models/svm_panstarrs_quasarsep_balanced.joblib')
smodel = joblib.load('models/svm_sdss_quasarsep_balanced_new.joblib')

cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel}), ('parallax_nan_edr3',),
             ('sdss_uvcut', {'model': smodel}), ('bayestar_chisq', {'maxchisq': 3.0})]

final_cut = combine_cuts(df_goodspec, cuts_list) #intersection of all cuts
df16a = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(df16a['sdss_dr17_specobj.CLASS'])

Cut0 62945 0.9628003732199397
Cut1 61712 0.9439405295440293
Cut2 55258 0.8452207963045107
Cut3 65092 0.9956406687367116
Cut4 55330 0.84632210104471
Cut5 52118 0.7971916729124922
Effective Cut = 0.696
Number of Stars = 45473, Fraction(%) = 99.8989, Ratio(obj:star)*100 = 100
Number of Galaxies = 30, Fraction(%) = 0.0659, Ratio(obj:star)*100 = 0.0660
Number of QSOs = 16, Fraction(%) = 0.0352, Ratio(obj:star)*100 = 0.0352


### Ablation Tests

In [5]:
df_goodspec = pd.read_pickle('specmatched_southrerun.pkl')
df_goodspec['chisq'] = df_goodspec['chisq_rerun']

In [6]:
spec_utils.print_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'])

Number of Stars = 55474, Fraction(%) = 84.8525, Ratio(obj:star)*100 = 100
Number of Galaxies = 87, Fraction(%) = 0.1331, Ratio(obj:star)*100 = 0.1568
Number of QSOs = 9816, Fraction(%) = 15.0145, Ratio(obj:star)*100 = 17.6948


In [10]:
wisemodel = joblib.load('models/svm_panstarrs_quasarsep_balanced.joblib')
smodel = joblib.load('models/svm_sdss_quasarsep_balanced_new.joblib')

In [7]:
nbtest_dir = '../../notebooks_test/'

def combine_cuts(df_input, cuts_list):
    cutfuncs = [getattr(astro_cuts, ctup[0]) for ctup in cuts_list]
    masklist = []
    for c, ctup in enumerate(cuts_list):
        if len(ctup)==1:
            masklist.append(cutfuncs[c](df_input))
        else:
            masklist.append(cutfuncs[c](df_input, **ctup[1]))
        print('Cut{} {} {}'.format(c, np.sum(masklist[c]), np.sum(masklist[c])/len(masklist[c])))
    final_cut = reduce(np.logical_and, masklist) #intersection of all cuts
    print('Effective Cut = {:.3f}'.format(np.sum(final_cut)/len(final_cut)))
    return final_cut

In [8]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}) ]

final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 62945 0.9628003732199397
Cut1 61712 0.9439405295440293
Effective Cut = 0.907
Number of Stars = 50205, Fraction(%) = 84.6899, Ratio(obj:star)*100 = 100
Number of Galaxies = 84, Fraction(%) = 0.1417, Ratio(obj:star)*100 = 0.1673
Number of QSOs = 8992, Fraction(%) = 15.1684, Ratio(obj:star)*100 = 17.9106
StarFracChange = -0.163 (%)
GalFracChange = 0.009 (%)
QSOFracChange = 0.154 (%)
Stars thrown out = -5269


In [11]:
cuts_list =  [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel})]

final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 62945 0.9628003732199397
Cut1 61712 0.9439405295440293
Cut2 55258 0.8452207963045107
Effective Cut = 0.765
Number of Stars = 49464, Fraction(%) = 98.9478, Ratio(obj:star)*100 = 100
Number of Galaxies = 58, Fraction(%) = 0.1160, Ratio(obj:star)*100 = 0.1173
Number of QSOs = 468, Fraction(%) = 0.9362, Ratio(obj:star)*100 = 0.9461
StarFracChange = 14.095 (%)
GalFracChange = -0.017 (%)
QSOFracChange = -14.078 (%)
Stars thrown out = -6010


In [12]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel}), ('parallax_nan_edr3',)]

final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 62945 0.9628003732199397
Cut1 61712 0.9439405295440293
Cut2 55258 0.8452207963045107
Cut3 65092 0.9956406687367116
Effective Cut = 0.762
Number of Stars = 49305, Fraction(%) = 98.9643, Ratio(obj:star)*100 = 100
Number of Galaxies = 48, Fraction(%) = 0.0963, Ratio(obj:star)*100 = 0.0974
Number of QSOs = 468, Fraction(%) = 0.9394, Ratio(obj:star)*100 = 0.9492
StarFracChange = 14.112 (%)
GalFracChange = -0.037 (%)
QSOFracChange = -14.075 (%)
Stars thrown out = -6169


In [13]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel}), ('parallax_nan_edr3',),
             ('sdss_uvcut', {'model': smodel})]

final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 62945 0.9628003732199397
Cut1 61712 0.9439405295440293
Cut2 55258 0.8452207963045107
Cut3 65092 0.9956406687367116
Cut4 55330 0.84632210104471
Effective Cut = 0.751
Number of Stars = 48998, Fraction(%) = 99.8187, Ratio(obj:star)*100 = 100
Number of Galaxies = 41, Fraction(%) = 0.0835, Ratio(obj:star)*100 = 0.0837
Number of QSOs = 48, Fraction(%) = 0.0978, Ratio(obj:star)*100 = 0.0980
StarFracChange = 14.966 (%)
GalFracChange = -0.050 (%)
QSOFracChange = -14.917 (%)
Stars thrown out = -6476


In [14]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel}), ('parallax_nan_edr3',),
             ('sdss_uvcut', {'model': smodel}), ('bayestar_chisq', {'maxchisq': 3.0})]

final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 62945 0.9628003732199397
Cut1 61712 0.9439405295440293
Cut2 55258 0.8452207963045107
Cut3 65092 0.9956406687367116
Cut4 55330 0.84632210104471
Cut5 52118 0.7971916729124922
Effective Cut = 0.696
Number of Stars = 45473, Fraction(%) = 99.8989, Ratio(obj:star)*100 = 100
Number of Galaxies = 30, Fraction(%) = 0.0659, Ratio(obj:star)*100 = 0.0660
Number of QSOs = 16, Fraction(%) = 0.0352, Ratio(obj:star)*100 = 0.0352
StarFracChange = 15.046 (%)
GalFracChange = -0.067 (%)
QSOFracChange = -14.979 (%)
Stars thrown out = -10001


### All cuts except

WISE

In [15]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('parallax_nan_edr3',), ('sdss_uvcut', {'model': smodel}), 
             ('bayestar_chisq', {'maxchisq': 3.0})]

final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 62945 0.9628003732199397
Cut1 61712 0.9439405295440293
Cut2 65092 0.9956406687367116
Cut3 55330 0.84632210104471
Cut4 52118 0.7971916729124922
Effective Cut = 0.701
Number of Stars = 45731, Fraction(%) = 99.8515, Ratio(obj:star)*100 = 100
Number of Galaxies = 31, Fraction(%) = 0.0677, Ratio(obj:star)*100 = 0.0678
Number of QSOs = 37, Fraction(%) = 0.0808, Ratio(obj:star)*100 = 0.0809
StarFracChange = 14.999 (%)
GalFracChange = -0.065 (%)
QSOFracChange = -14.934 (%)
Stars thrown out = -9743


Reduced Chisq

In [16]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel}), ('parallax_nan_edr3',),
             ('sdss_uvcut', {'model': smodel})]


final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 62945 0.9628003732199397
Cut1 61712 0.9439405295440293
Cut2 55258 0.8452207963045107
Cut3 65092 0.9956406687367116
Cut4 55330 0.84632210104471
Effective Cut = 0.751
Number of Stars = 48998, Fraction(%) = 99.8187, Ratio(obj:star)*100 = 100
Number of Galaxies = 41, Fraction(%) = 0.0835, Ratio(obj:star)*100 = 0.0837
Number of QSOs = 48, Fraction(%) = 0.0978, Ratio(obj:star)*100 = 0.0980
StarFracChange = 14.966 (%)
GalFracChange = -0.050 (%)
QSOFracChange = -14.917 (%)
Stars thrown out = -6476


SDSS

In [17]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel}), ('parallax_nan_edr3',),
             ('bayestar_chisq', {'maxchisq': 3.0})]


final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 62945 0.9628003732199397
Cut1 61712 0.9439405295440293
Cut2 55258 0.8452207963045107
Cut3 65092 0.9956406687367116
Cut4 52118 0.7971916729124922
Effective Cut = 0.699
Number of Stars = 45612, Fraction(%) = 99.7747, Ratio(obj:star)*100 = 100
Number of Galaxies = 32, Fraction(%) = 0.0700, Ratio(obj:star)*100 = 0.0702
Number of QSOs = 71, Fraction(%) = 0.1553, Ratio(obj:star)*100 = 0.1557
StarFracChange = 14.922 (%)
GalFracChange = -0.063 (%)
QSOFracChange = -14.859 (%)
Stars thrown out = -9862


Parallax nan edr3

In [18]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel}),
             ('sdss_uvcut', {'model': smodel}), ('bayestar_chisq', {'maxchisq': 3.0})]


final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 62945 0.9628003732199397
Cut1 61712 0.9439405295440293
Cut2 55258 0.8452207963045107
Cut3 55330 0.84632210104471
Cut4 52118 0.7971916729124922
Effective Cut = 0.699
Number of Stars = 45616, Fraction(%) = 99.8905, Ratio(obj:star)*100 = 100
Number of Galaxies = 34, Fraction(%) = 0.0745, Ratio(obj:star)*100 = 0.0745
Number of QSOs = 16, Fraction(%) = 0.0350, Ratio(obj:star)*100 = 0.0351
StarFracChange = 15.038 (%)
GalFracChange = -0.059 (%)
QSOFracChange = -14.979 (%)
Stars thrown out = -9858
